# Homework 4: Problem 2

**Part A [30 points]** *Note this is a Collaborative Problem*

Using the Gaussian kernel develop psuedo code to create a SVM system to accomplish the
following steps:

+ Develop the ability to read in data xn with n observations and D dimensions (number of features).
+ Develop the ability to randomly remove 20% of the observations per class and assign the observations as test data with the remaining 80% of the observations as training data.
+ Using the equations in the Machine Learning I document under the Support Vector Machine section to develop an algorithm to process an input observations and compare it with the training observations.
+ Expand the development to handle multiple classes. The SVM is a two class classifier so it is recommended to use one class vs. the others. This will require multiple models to be developed based on the number of classes.

**Part B [20 points]** *Note this is a Collaborative Problem*
+ Calculate the running time of the system above in O-notation.
+ Calculate the total running time of the above system as T(n) with each line of pseudocode or code accounted for.
+ How does the total running time T(n) compare to the running time in O-notation?

## Data Load

In all operations in which I read in Iris data for this course, I have leveraged the `load(0)` function of my [`Reader`](https://github.com/choct155/en685_621/blob/master/algorithms/iris/Reader.py) class. The function just leverages the fact that scikit-learn already has the Iris dataset, so the load occurs in constant time. The remaining work simply involves splitting the data for downstream use.

```python
class IrisReader:

    def __init__(self):
        super().__init__()

    def load(self):
        iris_in: np.array = datasets.load_iris()
        self.data = dict(
            setosa = iris_in["data"][:50],
            versicolor = iris_in["data"][50:100],
            virginica = iris_in["data"][100:]
        )
```

In effect, this function consists of two constant time operations for our purposes, yielding a recurrence of $2T(1)$ and $O(1)$ asymptotics.

In [92]:
import sklearn as skl
from sklearn.svm import SVC
from algorithms.iris.Reader import IrisReader
from algorithms.iris.IrisOps import IrisOps
from algorithms.classifiers.svm import SupportVector
from typing import Dict, Tuple, Callable, List
import numpy as np
from functools import reduce

iris_reader: IrisReader = IrisReader()
iris_reader.load()
raw_data: Dict[str, np.array] = iris_reader.data

raw_data

{'setosa': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],


## Split into Test (20%) - Train (80%)

While **`scikit-learn`** does have it's own train-test-split function, the shape of my data after the load does not precisely play nice with it. For this reason, I have another `test_train_split()` function defined within my [`IrisOps`](https://github.com/choct155/en685_621/blob/master/algorithms/iris/IrisOps.py) class. It combines the output of `Reader.load()` across classes, randomly permutes the data, and then yields a tuple containing the train and test sets, respectively. It also labels the data with an index value corresponding to each class.

```python
def test_train_split(raw_data: Dict[str, np.array], labels: List[str], train_prop: float) -> Tuple[np.array, np.array]:
    
    def process_label_group(data: np.array, idx: int) -> np.array:
        n: int = len(data)
        lab_idx: np.array = np.repeat(idx, n).reshape(n, 1)
        return np.concatenate([lab_idx, data], axis=1)

    print("Label Mapping: ", list(enumerate(labels)))
    data: np.array = np.concatenate(list(
        map(lambda lab: process_label_group(raw_data[lab[1]], lab[0]), enumerate(labels))
    ))
    permuted: np.array = np.random.permutation(data)

    train_n: int = int(len(permuted) * 0.8)
    return (permuted[:train_n], permuted[train_n:])
```

Assigning a label (see the helper `process_label_group()`) involves an assignment ($T(1)$), the instantiation of an array (\~ $T(n)$), and the columnwise concatenation of two equivalently long arrays (\~$T(n)$). We can ignore the reshaping of the array, which is effectively a metadata exercise. The helper function has a total cost of $2T(n) + 1$ if $n$ denotes the size of the array that is labeled. 

However, we should note that the `process_label_group()` only operates on a third of the data at a time in the Iris case, but since it still must do all three, the recurrence should hold. Once all three species have been labeled, they must be concatenated. When I have written concatenation operations in the past, I tend to use a fold with the first array as the starting value and appending additional value with each subsequent recursive call. So, a recurrence of $T(\frac{2n}{3})$ seems fitting here.

The next step is a permutation which likely involves sampling (indices) without replacement and then a sort. Let us assume we can deplete a collection in linear time and assume a sort reminiscient of merge-sort ($2T(\frac{n}{2}) + O(n)$) for a total run time of $2T(\frac{n}{2}) + 2T(n)$. 

Since we are dealing with arrays (which we will assume aren't Lists, or Vectors, or something under the hood), we should have constant time indexing which I suspect supports constant time splits. So, our last operation, splitting the data into the test and train sets adds $T(1)$.

Altogether, the helper function, concatenation, permutation, and split have th following cost:

\begin{align}
    T(n) &= 2T(n) + 1 + T(\frac{2n}{3}) + 2T(\frac{n}{2}) + 2T(n) + 1 \\
    &= 4T(n) + T(\frac{2n}{3}) + 2T(\frac{n}{2}) + 2
\end{align}

Since none of these terms are interacting with each other in a nested loop, the asymptotic growth is capped at $O(n)$.

##  Develop a Support Vector Machine Classifier with a Gaussian Kernel

*Note: I initially set out to construct the full implementation from scratch, but I found I got a little hung up on the optimization portion. In the interests of time, I opted for using the `scikit-learn` implementation. I'll include the failed implementation in an Appendix to this notebook, just in case your curiosity gets the better of you. I have no expectation, but if you do feel inclined to provide a pointer or two, I wouldn't be upset about it.*

The estimator used in this notebook is provided by the `SupportVector` class in my [`svm`](https://github.com/choct155/en685_621/blob/master/algorithms/classifiers/svm.py) module. It leverages the `scikit-learn` classifier under the hood, which is capable of multi-class assignment. However, to adhere to the spirit of the assignment, this class leverages that implementation as a binary classifier, and then uses a round robin approach to collect labels which are then reconciled at the end.

### SVM Psuedo-Code Analysis

Since a stock approach is leveraged below, let's evaluate a psuedo-code version of the SVM approach. The focus here is on the dual soft margin SVM because it exposes an isolated inner product for the feature matrix, which creates an opportunity for kernel trickery. For the dual soft margin SVM our task is to minimize the following relation with respect to $\alpha$.

\begin{align}
    L(\alpha) &= \Sigma_{n=1}^N - \frac{1}{2} \Sigma_{n=1}^N \Sigma_{m=1}^N a_n a_m y_n y_m k(\textbf{x_n}, \textbf{x_m}) \\
    &= \alpha^T - \frac{1}{2} \alpha^T Q \alpha \\
\end{align}
where $Q_{N \times N} = y_n^T y_m * K(\textbf{x_n}, \textbf{x_m})$ and subject to $0 \leq \alpha_n \leq C \hspace{3pt} \forall \hspace{3pt} n$ and $\alpha^T y = 0$. The matrix $K(\textbf{x_n}, \textbf{x_m})$ is commonly known as the Gram Matrix, and is the insertion point for the gaussian kernel (or any other kernel we might choose). As discussed in Problem 1, the construction of the Gram Matrix costs a running time of $T(mn) + 3$ with an asymptotic cost of $O(mn)$ where $m$ is the number of test observations and $n$ is the number of train observations. In this instance, we are actually calculating the inner product of the train data with itself, so we are looking at $T(n^2) + 3$ and $O(n^2)$.

The problem can mostly be split in two: setting up for the optimization used to discover the weights and bias, and then solving the optimization. The last part, label assignment, is just an $T(n)$ map over the test data that leverages the weights and bias identified by optimization.

#### Set Up

1. Compute the Gram Matrix ($T(n^2) + T(3)$)
2. Compute inner product of labels and use it to scale the Gram Matrix ($T(n) + T(1)$)
3. Construct initial guess for matrix of Lagrangian multipliers $\alpha_n$ ($T(1)$)
4. Construct bounds for the inequality constraints by concatenating two vectors of length $n$, populated with values of 0 and $C$, respectively. Additionally, concatenate two matrices to facilitate conformal comparison to the bounds, which is more costly if simple appends are used. ($T(n) + T(n^2)$; see problem 1 for discussion of concatenation)
5. Assign existing labels for the equality constraint ($T(1)$), and the value of zero ($T(1)$)

Total set up cost is $2T(n^2) + 2T(n) + T(7)$.

#### Optimization

I am a bit unclear about how to appraise the running cost of a non-deterministic algorithm like stochastic gradient descent, or even one that is so sensitive to starting conditions like base gradient descent. It strikes me that I can at least say a computational challenge is the need to solve the likelihood at each step for all dimensions of the data to determine the direction of greatest ascent (in order to move in the opposite direction). That is, optimization scales at $O(d)$ where $d$ is the dimensionality of the data to be classified.

The total running cost of set up, "optimization", and labeling is therefore $2T(n^2) + 3T(n) + T(7) + O(d)$. When looking at the asymptotic cost, it is unclear whether or not the observation count or the dimensionality count will be dominant in all cases, so we can cover both bases with $O(n^2) + O(d)$. Running time again exceeds asymptotic time.

### Implementation of Multi-Class Assignment via Binary Classification

The approach taken in the `SupportVector` class is as follows:

For each class: 
1. Alter the labels to assign +1 to the selected class and -1 to the other two in the train data
2. Train SVM classifier on train data
3. Predict labels on test data
4. Assign the selected class to labels with values of +1, and "other" to labels with values of -1

Once each class has a vector of labels, filter out "other" from each row and compare to the truth vector. If the match is exact, it counts as a match. If there are conflicting matches across runs, it counts as a mismatch.

In [98]:
sv = SupportVector(raw_data)
rbf = sv.fit([1,2,3,4], sv.train)
rbf

Label Mapping:  [(0, 'setosa'), (1, 'versicolor'), (2, 'virginica')]


SVC(C=1, gamma=0.5)

In [99]:
results = SupportVector.binary_round_robin(sv.labels, [1,2,3,4], sv.train, sv.test)

results

{'accuracy': 1.0,
 'predictions': [array(['versicolor'], dtype='<U10'),
  array(['setosa'], dtype='<U10'),
  array(['setosa'], dtype='<U10'),
  array(['setosa'], dtype='<U10'),
  array(['setosa'], dtype='<U10'),
  array(['versicolor'], dtype='<U10'),
  array(['virginica'], dtype='<U10'),
  array(['virginica'], dtype='<U10'),
  array(['setosa'], dtype='<U10'),
  array(['setosa'], dtype='<U10'),
  array(['virginica'], dtype='<U10'),
  array(['versicolor'], dtype='<U10'),
  array(['setosa'], dtype='<U10'),
  array(['setosa'], dtype='<U10'),
  array(['setosa'], dtype='<U10'),
  array(['virginica'], dtype='<U10'),
  array(['setosa'], dtype='<U10'),
  array(['versicolor'], dtype='<U10'),
  array(['virginica'], dtype='<U10'),
  array(['virginica'], dtype='<U10'),
  array(['virginica'], dtype='<U10'),
  array(['virginica'], dtype='<U10'),
  array(['versicolor'], dtype='<U10'),
  array(['virginica'], dtype='<U10'),
  array(['virginica'], dtype='<U10'),
  array(['setosa'], dtype='<U10'),
  array

In practice, there have been no conflicting labels.

#  Appendix - Stalled Attempt at Ground Up Implementation (for now)

For the dual soft margin SVM our task is to minimize the following relation with respect to $\alpha$.

\begin{align}
    L(\alpha) &= \Sigma_{n=1}^N - \frac{1}{2} \Sigma_{n=1}^N \Sigma_{m=1}^N a_n a_m y_n y_m k(\textbf{x_n}, \textbf{x_m}) \\
    &= \alpha^T - \frac{1}{2} \alpha^T Q \alpha \\
\end{align}
where $Q_{N \times N} = y_n^T y_m * K(\textbf{x_n}, \textbf{x_m})$ and subject to $0 \leq \alpha_n \leq C \hspace{3pt} \forall \hspace{3pt} n$ and $\alpha^T y = 0$. The matrix $K(\textbf{x_n}, \textbf{x_m})$ is commonly known as the Gram Matrix, and is the insertion point for the gaussian kernel (or any other kernel we might choose).

In [100]:
def gram_matrix(X: np.array, Y: np.array, spread: float) -> np.array:
    x_rows, x_cols = X.shape
    y_rows, y_cols = Y.shape
    out: np.array = np.zeros((x_rows, y_rows))
    
    def g(x_0: np.array, x_n: np.array) -> float:
        normalization: float = 1 / ((np.sqrt(2*np.pi)*spread)**len(x_0))
        distance: float = (x_0-x_n).dot((x_0-x_n))
        exponential: float = np.exp((-0.5/spread**2) * distance)
        return normalization * exponential
    
    for i, x in enumerate(X):
        for j, y in enumerate(Y):
            out[i, j] = g(x, y)
            
    return out

Finding the optimal solution of $\alpha$ is actually a quadratic programming problem, so we will use a solver from an optimization library called [CVXOPT](https://cvxopt.org/userguide/intro.html). Specifically, we will use the [`qp`](https://cvxopt.org/userguide/coneprog.html#quadratic-programming) solver because it is designed for such problems.

In [101]:
%%html
<iframe src="https://cvxopt.org/userguide/coneprog.html#quadratic-programming" width="1200" height="1000"></iframe>

To understand what we are doing here, let's walk through each of the arguments and see how they correspond.

+ `x` is the parameter to be optimized over, so it is $\alpha$
+ `P` is our scaled Gram Matrix $Q_{N \times N} = y_n^T y_m * K(\textbf{x_n}, \textbf{x_m})$
+ `q` appears to be our initial guess for $\alpha$
+ Our first constraint has to be reworked a bit to fit the shape of the API (see $Gx \leq h$). Instead of expressing the bounds on $\alpha$ as the interval $0 \leq \alpha_n \leq C$, we will break it up into two expressions: $-\alpha_n \leq 0$ and $\alpha_n \leq C$. From here, we can stack the two composite constraints together.
    + `h` is the vertical concatenation of two $N \times 1$ vectors. The first is filled with zeros, capturing the lower bound of our initial constraint. The second is filled with values of $C$, the upper bound of the initial constraint. 
    + `G` is the cofficient matrix that enables us to compare each value of `x` (i.e. $\alpha$) to the constraints. In this case, we just want a direct value comparison, so we will stack two identity matrices on top of one another. Note, that each corresponds to each half of our initial constraint, so we must multiply the upper identity matrix by -1 to test $-\alpha_n \leq 0$.
+ `A` plays the role of the labels $y$ in the second constraint $\alpha^T y = 0$, and `b` plays the role of the zero.

In [102]:
def test_train_split(raw_data: Dict[str, np.array], labels: List[str], train_prop: float) -> Tuple[np.array, np.array]:
    
    def process_label_group(data: np.array, idx: int) -> np.array:
        n: int = len(data)
        lab_idx: np.array = np.repeat(idx, n).reshape(n, 1)
        return np.concatenate([lab_idx, data], axis=1)
    
    print("Label Mapping: ", list(enumerate(labels)))
    data: np.array = np.concatenate(list(
        map(lambda lab: process_label_group(raw_data[lab[1]], lab[0]), enumerate(labels))
    ))
    permuted: np.array = np.random.permutation(data)
    
    train_n: int = int(len(permuted) * 0.8)
    return (permuted[:train_n], permuted[train_n:])
    
train, test = test_train_split(raw_data, ["setosa", "versicolor", "virginica"], .8)

Label Mapping:  [(0, 'setosa'), (1, 'versicolor'), (2, 'virginica')]


In [103]:
def process_data(data: np.array, pos_y_class: int) -> Tuple[np.array, np.array]:
    y: np.array = np.where(data[:,0] == pos_y_class, 1, -1)
    x_in: np.array = data[:, 1:]
    X: np.array = (x_in - x_in.mean(axis=0)) / x_in.std(axis=0)
    return y, X
    
y_train, X_train = process_data(train, 0)

In [104]:
from cvxopt import matrix, solvers
def train(y, X, margin_error, bandwidth):
    n_rows, n_cols = X.shape
    y_2d: np.array = y.reshape(1, n_rows).astype(float)
    
    P: np.array = -np.dot(y_2d.T, y_2d) * gram_matrix(X, X, bandwidth)
    q: np.array = np.ones((n_rows, 1))
    h: np.array = np.concatenate([
        np.zeros((n_rows, 1)),
        np.ones((n_rows, 1)) * margin_error
    ], axis=0)
    G: np.array = np.concatenate([
        -np.eye((n_rows)),
        np.eye((n_rows))
    ], axis=0)
    A: np.array = y_2d
    b: np.array = np.zeros(1)
        
    for elem in [P, q, h, G, A, b]:
        print(np.ndim(elem))
        
    solvers.options['abstol'] = 1e-10
    solvers.options['reltol'] = 1e-10
    solvers.options['feastol'] = 1e-10
    
    return solvers.qp(
        matrix(P), 
        matrix(q), 
        matrix(G), 
        matrix(h), 
        matrix(A), 
        matrix(b)
    )

train(y_train, X_train, 10, 0.3)

2
2
2
2
2
1


ValueError: Rank(A) < p or Rank([P; A; G]) < n

In [105]:
??matrix